In [7]:
import numpy as np
from RandomForest import RandomForestSOMAC

In [8]:
train_file = "/Users/andre.gomes/Temp/SOMAC-ML/data/28082018/backlog_file.npy"
test_file = "/Users/andre.gomes/Temp/SOMAC-ML/data/14092018/somac_1/backlog_file.npy"

In [9]:
test_data = np.load(test_file, encoding = "latin1").item()

In [10]:
rf = RandomForestSOMAC()
rf.train(train_file)

In [11]:
for t in range(80):
    rf.decision(test_data[t])
    print("---")

Confidence Interval = [0.40317971], [0.76129857]
y_hat_sorted mean = 0.57
Confidence Interval = [0.70880966], [1.58563204]
y_hat_sorted mean = 1.27
Prot: 0, y = 0.0
y_hat_CSMA = 0.58, y_hat_TDMA = 1.15
Evaluation: v_csma = 0.58, v_tdma = 1.15
v_csma_ = 21, v_tdma_ = 29
---
Confidence Interval = [1.01159013], [1.5301693]
y_hat_sorted mean = 1.41
Confidence Interval = [2.55935498], [3.69750975]
y_hat_sorted mean = 3.27
Prot: 1, y = 0.9482759833335876
y_hat_CSMA = 1.27, y_hat_TDMA = 3.13
Evaluation: v_csma = 4.21, v_tdma = 7.29
v_csma_ = 9, v_tdma_ = 41
---
Confidence Interval = [1.07686918], [1.65054748]
y_hat_sorted mean = 1.52
Confidence Interval = [2.88095245], [4.00638918]
y_hat_sorted mean = 3.66
Prot: 1, y = 2.6333330869674683
y_hat_CSMA = 1.36, y_hat_TDMA = 3.44
Evaluation: v_csma = 5.07, v_tdma = 7.15
v_csma_ = 7, v_tdma_ = 43
---
Confidence Interval = [1.04598892], [1.55428617]
y_hat_sorted mean = 1.44
Confidence Interval = [2.63239117], [3.76026207]
y_hat_sorted mean = 3.36
Pro

Confidence Interval = [16.53193667], [18.17546049]
y_hat_sorted mean = 17.17
Confidence Interval = [15.58013795], [17.17704306]
y_hat_sorted mean = 16.87
Prot: 1, y = 21.692529022693634
y_hat_CSMA = 17.35, y_hat_TDMA = 16.38
Evaluation: v_csma = 24.17, v_tdma = 17.9
v_csma_ = 31, v_tdma_ = 19
No. of new estimators = 5
---
Confidence Interval = [16.53193667], [18.17546049]
y_hat_sorted mean = 17.17
Confidence Interval = [16.74466468], [18.20455636]
y_hat_sorted mean = 17.48
Prot: 1, y = 21.906896233558655
y_hat_CSMA = 17.35, y_hat_TDMA = 17.47
Evaluation: v_csma = 24.19, v_tdma = 18.98
v_csma_ = 24, v_tdma_ = 26
---
Confidence Interval = [16.54400212], [18.19336148]
y_hat_sorted mean = 17.17
Confidence Interval = [16.74466468], [18.20455636]
y_hat_sorted mean = 17.48
Prot: 1, y = 24.329883694648743
y_hat_CSMA = 17.37, y_hat_TDMA = 17.47
Evaluation: v_csma = 24.23, v_tdma = 18.97
v_csma_ = 24, v_tdma_ = 26
---
Confidence Interval = [16.56279906], [18.11704865]
y_hat_sorted mean = 17.22
C